In [7]:
import pandas as pd
import numpy
import xlrd
import re
import glob
import xlrd
       
def process_timesheet(unzipped_datafile, timesheet):
    workbook = xlrd.open_workbook(unzipped_datafile)
    sheet = workbook.sheet_by_name('Timesheet')
    
    #Process first tab of timesheet
    #On the first tab, save cell B3 as the contractor name
    name = sheet.cell_value(2, 1)
    #On the first tab, save cell B8 as the contractor_id
    contractor_id = str(sheet.cell_value(3,4))[:3]
        #On the first tab, save cell D18 as the contractor rate
    payrate = sheet.cell_value(17,3)
    if payrate == "Hourly \nRate":
        payrate = sheet.cell_value(18,3)  #likely a more elegant solution, but brute forcing for now
    date = re.search('\d+[-]\d+[-]\d+', unzipped_datafile).group(0)

    #Process second tab of Excel spreadsheet
    sheet2 = workbook.sheet_by_name('Task Description')
    #On the second tab, starting with cell A6:E6, save each row until you encounter an empty row
    row = 5
    hours = sheet2.cell_value(row,0)
    activity = sheet2.cell_value(row, 1)
    description = sheet2.cell_value(row, 4)
    budget_category = sheet2.cell_value(row,8)
    while hours != '' and activity != "Total":
        cost = payrate * hours
        timesheet = timesheet.append({'Name':name, 'Contractor_ID': contractor_id, 'Payrate':payrate, 'Invoice Date':date, 'Hours':hours, 'Cost': cost, 'Activity':activity, 'Description':description, 'Budget Category': budget_category}, ignore_index=True)
        row += 1
        hours = sheet2.cell_value(row,0)
        activity = sheet2.cell_value(row, 1)
        description = sheet2.cell_value(row, 4)
        budget_category = sheet2.cell_value(row, 8)
    return timesheet, date

def scrub_budget_category(category):
    if re.search('([Ss]tandard)', category) is not None:
        category = "Standards Collections"
    if re.search('([Aa]ctivation)', category) is not None:
        category = "Navigate Activation"
    if re.search('([Mm]aint)', category) is not None:
        category = "Navigate Content - Maintenance"
    if re.search('([Nn]ew [Dd]ev)', category) is not None:
        category = "Navigate Content - New Development"
    if re.search('([Nn]ew [Pp]rod)', category) is not None:
        category = "Navigate Content - New Product Development"
    if re.search('([Nn]ew [Ii]tem)', category) is not None:
        category = "Navigate Content - New Development"
    return category

#open template file to write summary values

from openpyxl import load_workbook

def create_budget(totals, date):
    #dictionary of column indices for budget categories as found in budget template
    categories = {'Navigate Content - New Development': 7, 
                  'Navigate Content - Maintenance': 8, 
                  'Navigate Content - New Product Development': 9,
                  'Navigate Activation' : 10,
                  'Standards Collections': 11}
    filename = "Budget breakdown template.xlsx"
    #Open the budget template file
    wb = load_workbook(filename = filename)
    #Get the current Active Sheet
    ws = wb.active
    row = 0
    while row < len(totals):
        #find row in budget template for contractor_id
        contractor_id = totals.iloc[row]['Contractor_ID']
        #print contractor_id
        budget_category = totals.iloc[row]['Budget Category']
        #print budget_category
        try:
            budget_column = categories[budget_category]
        except:
            print "Program stopped: Contractor " + contractor_id + " has errors in budget category entry: \"" + budget_category + "\""
            break
        #print budget_column
        cost = totals.iloc[row]['Cost']
        #print cost
        i=1
        while i < 40:
            #print ws.cell(row=i, column=3).value
            if ws.cell(row=i, column =3).value == contractor_id:
                ws.cell(row=i, column=budget_column).value = cost
                #print "HI!"
                break
            else:
                i += 1
        row += 1
    
    #save the csv file
    filename = "Auto-Budget breakdown for " + date + ".xlsx"
    wb.save(filename)


In [8]:
if __name__ == "__main__":
    timesheet = pd.DataFrame([],columns=['Name', 'Contractor_ID', 'Invoice Date','Payrate','Hours', 'Cost', 'Activity',\
                                         'Description', 'Budget Category'])
    #apply process_timesheet to all unzipped Excel files
    for file in glob.glob("*.xls*"):
        if (re.search('budget', file) == None) and (re.search('Budget', file) == None):
            timesheet, date = process_timesheet(file, timesheet)
    timesheet['Budget Category'] = timesheet['Budget Category'].apply(scrub_budget_category)
    category_totals = timesheet.groupby('Budget Category').sum()
    budget = timesheet[['Contractor_ID', 'Budget Category', 'Cost']]
    individual_totals_by_budget = budget.groupby(['Contractor_ID', 'Budget Category']).sum().reset_index()
    create_budget(individual_totals_by_budget, date)